In [1]:
import os
import pathlib
import math
import itertools
import pandas as pd
import numpy as np
from negmas import *
from negmas.apps.scml import *
from negmas.apps.scml import Input, Output, Job, FactoryStatusUpdate
from negmas.situated import Contract

ModuleNotFoundError: No module named 'negmas.apps'

In [ ]:
# setup disply parameters
%pylab inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%config InlineBackend.figure_format = 'retina'
VARY_SMALL_SIZE = 24
SMALL_SIZE = 36
MEDIUM_SIZE = 48
BIGGER_SIZE = 64

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=VARY_SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
pylab.rcParams['figure.figsize'] = (24, 6)
plt.rc('lines', linewidth=3)

In [ ]:
n_steps = 10
horizon = 4
world = SCMLWorld.tiny(n_intermediate_levels=0, log_file_name='', n_miners=2, n_consumers=2, n_factories_per_level=2, n_steps=n_steps, default_signing_delay=0
                       , allow_immediate_negotiations=False, consumer_kwargs={'immediate_cfp_update': True
        , 'consumption_horizon': horizon}, negotiation_speed_multiplier=None, neg_n_steps=5)
world.run()
data=None
if len(world.saved_contracts) > 0:
    data = pd.DataFrame(data=world.saved_contracts).loc[:, ["buyer", "seller", "product", "quantity", "delivery_time"
        , "unit_price", "penalty", "signing_delay", "concluded_at", "signed_at", "issues", "cfp"]]
data

In [ ]:
pd.DataFrame(data=world.stats)